In [2]:
import gym


In [6]:
env = gym.make("CartPole-v1")
print(env.action_space)
print(env.observation_space)

observation = env.reset()
print("Initial observations:", observation)

observation, reward, done, info, extra_info = env.step(0)

print("New observations after choosing action 0:", observation)
print("Reward for this step:", reward)
print("Is this round done?", done)

observation = env.reset()
cumulative_reward = 0
done = False
while not done:
    observation, reward, done, info, extra_info = env.step(0)
    cumulative_reward += reward
print("Cumulative reward for this round:", cumulative_reward)

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Initial observations: (array([-0.01628398, -0.0382522 , -0.04780297, -0.04398627], dtype=float32), {})
New observations after choosing action 0: [-0.01704903 -0.23265724 -0.04868269  0.23323946]
Reward for this step: 1.0
Is this round done? False
Cumulative reward for this round: 9.0


c:\Users\kentp\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [ ]:
class 

### Task 1:  Development of an RL Agent

In [ ]:
from collections import defaultdict
import numpy as np


class Monte_Carlo():
    def __init__(self, env) -> None:
        self.env = env
        self.q_table = defaultdict(int)
        self.r_table = defaultdict(int)
        self.discount_factor = 0.9 # test
        self.epsilon = 0.1 # test 
    
    def discretise_state(self, state: np.ndarray) -> np.ndarray:
        state[0] = round(state[0], 1)
        state[1] = round(state[1], 0)
        state[2] = round(state[2], 1)
        state[3] = round(state[3], 0)
        return state
    
    def generate_episode(self, policy) -> list:
        episode = []
        state = self.env.reset()
        done = False
        while not done:
            action = policy(state)
            next_state, reward, done, info_1, info_2 = self.env.step(action)
            episode.append((state, action, reward))
            state = next_state
        return episode

    def generate_return(self, episode) -> None:
        G = defaultdict(int)

        accumulated_reward = 0

        # Iterate backwards according to G(t) = r(t) + y * G(t+1)
        for i in reversed(range(len(episode))):
            state, action, reward = episode[i]
            accumulated_reward = reward + self.discount_factor * accumulated_reward
            G[(state, action)] += accumulated_reward
        
        for (state, action) in G:
            self.r_table[(state, action)].append(G[(state, action)])

        return
    
    def update_q_table(self) -> None:
        for (state, action) in self.r_table:
            self.q_table[(state, action)] = np.mean(self.r_table[(state, action)])

    def policy(self, state) -> int:
        q_values = [self.q_table.get((state, a), 0) for a in range(self.env.action_space.n)]
        
        if np.random.rand() < self.epsilon:
            action = np.random.choice(range(self.env.action_space.n))
        else:
            action = np.argmax(q_values)
        
        return action
